In [21]:
from crewai import Agent, Task, Crew

In [22]:
import os

# Set the environment variable for the Hugging Face token


In [23]:
from langchain_community.llms import HuggingFaceHub

In [93]:
!ollama

Usage:
  ollama [flags]
  ollama [command]

Available Commands:
  serve       Start ollama
  create      Create a model from a Modelfile
  show        Show information for a model
  run         Run a model
  stop        Stop a running model
  pull        Pull a model from a registry
  push        Push a model to a registry
  list        List models
  ps          List running models
  cp          Copy a model
  rm          Remove a model
  help        Help about any command

Flags:
  -h, --help      help for ollama
  -v, --version   Show version information

Use "ollama [command] --help" for more information about a command.


In [92]:
prompt = "Explain the significance of cybersecurity in the modern world."

# Generate a response using the LLM
response = llm(prompt)

# Print the response
print(response)

ConnectionError: (ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')), '(Request ID: ce7268c7-eeca-4eea-9612-dd5d5a7c57eb)')

In [33]:
import requests

headers = {"Authorization": f"Bearer {os.getenv('HUGGINGFACEHUB_API_TOKEN')}"}
response = requests.get("https://api-inference.huggingface.co/models/HuggingFaceH4/zephyr-7b-beta", headers=headers)
print(response.status_code, response.text)

200 {"_id":"653a4cbd66081db6fc588b21","id":"HuggingFaceH4/zephyr-7b-beta","sha":"892b3d7a7b1cf10c7a701c60881cd93df615734c","pipeline_tag":"text-generation","library_name":"transformers","private":false,"gated":false,"siblings":[],"safetensors":{"parameters":{"BF16":7241732096}},"tags":["transformers","pytorch","safetensors","mistral","text-generation","generated_from_trainer","conversational","en","dataset:HuggingFaceH4/ultrachat_200k","dataset:HuggingFaceH4/ultrafeedback_binarized","arxiv:2305.18290","arxiv:2310.16944","arxiv:2305.14233","arxiv:2310.01377","base_model:mistralai/Mistral-7B-v0.1","base_model:finetune:mistralai/Mistral-7B-v0.1","license:mit","model-index","autotrain_compatible","text-generation-inference","endpoints_compatible","region:us"],"cardData":{"tags":["generated_from_trainer"],"base_model":"mistralai/Mistral-7B-v0.1"}}


In [76]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True,
    llm=llm
)

In [77]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True,
    llm=llm
)

In [78]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True,
    llm=llm
)

In [79]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

In [80]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

In [81]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

In [82]:
from opentelemetry import trace
trace.set_tracer_provider(None)

2024-11-19 12:27:57,804 - 8482410304 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [83]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

2024-11-19 12:27:58,115 - 8482410304 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [86]:
result = crew.kickoff(inputs={"topic": "data privacy in AI"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on AI.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
You are Content Planner. You're working on planning a blog article about the topic: AI.You collect information that helps the audience learn something and make informed decisions. Your work is the basis for the Content Writer to write an article on this topic.
Your personal goal is: Plan engaging and factually accurate content on AITo give my best complete final answer to the task use the exact following format:

Thought: I now can give a great answer
Final Answer: my best complete final answer to the task.
Your final answer must be the great and the most complete as possibl

In [87]:
from IPython.display import Markdown
Markdown(result)

my best complete final answer to the task.
Your final answer must be the great and the most complete as possible, it must be outcome described.

I MUST use these formats, my job depends on it!
Current Task: Proofread the given blog post for grammatical errors and alignment with the brand's voice.

This is the expect criteria for your final answer: A well-written blog post in markdown format, ready for publication, each section should have 2 or 3 paragraphs. 
 you MUST return the actual complete content as the final answer, not a summary.

This is the context you're working with:
my best complete final answer to the task.
Your final answer must be the great and the most complete as possible, it must be outcome described.

I MUST use these formats, my job depends on it!
Current Task: 1. Use the content plan to craft a compelling blog post on AI.
2. Incorporate SEO keywords naturally.
3. Sections/Subtitles are properly named in an engaging manner.
4. Ensure the post is structured with an engaging introduction, insightful body, and a summarizing conclusion.
5. Proofread for grammatical errors and alignment with the brand's voice.


This is the expect criteria for your final answer: A well-written blog post in markdown format, ready for publication, each section should have 2 or 3 paragraphs. 
 you MUST return the actual complete content as the final answer, not a summary.

This is the context you're working with:
Planning Engaging and Factually Accurate Content on AI

As AI continues to evolve, it's crucial for businesses and individuals to stay up-to-date with the latest trends, key players, and noteworthy news in the industry. In this content plan, we'll prioritize these developments and explore their impact on various industries. Additionally, we'll identify our target audience, develop a detailed

Begin! This is VERY important to you, use the tools available and give your best Final Answer, your job depends on it!

Thought: 

When I read the content plan, I realized that AI is a rapidly growing technology that has the potential to revolutionize various industries. As a content writer, my role is to create compelling and insightful content that educates and informs the audience about AI. In this blog post, I will discuss the latest trends, key players, and noteworthy news in the AI industry. The post will be structured with an engaging introduction, insightful body, and a summarizing conclusion.

Begin! This is VERY important to you, use the tools available and give your best Final Answer, your job depends on it!

Thought: 

As AI continues to evolve, businesses and individuals are recognizing its potential benefits. From healthcare to finance, AI is transforming various industries. In this section, we'll explore the latest trends in AI and their impact on different industries.

Trend 1: Advancements in Natural Language Processing (NLP)

NLP is a subfield of AI that focuses on enabling computers to understand and generate human-like language. Recent advancements in NLP have